In [1]:
import pennylane as qml
from pennylane import numpy as np

In [2]:
observables = [
    qml.PauliX(0) @ qml.PauliY(1),
    qml.PauliX(0) @ qml.PauliZ(2)
]

### Conventional Measurements

In [3]:
dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev, interface='autograd')
def circuit1(weights):
    qml.StronglyEntanglingLayers(weights, wires=range(3))
    return qml.expval(observables[0])

@qml.qnode(dev, interface='autograd')
def circuit2(weights):
    qml.StronglyEntanglingLayers(weights, wires=range(3))
    return qml.expval(observables[1])

param_shape = qml.templates.StronglyEntanglingLayers.shape(n_layers=3, n_wires=3)
rng = np.random.default_rng(192933)
weights = rng.normal(scale=0.1, size=param_shape)

print("Expectation value of XYI =", circuit1(weights))
print("Expectation value of XIZ =", circuit2(weights))

Expectation value of XYI = 0.005259062416898108
Expectation value of XIZ = 0.1678258917705896


### QWC Single Measurement

In [8]:
@qml.qnode(dev, interface='autograd')
def circuit_qwc(weights):
    qml.StronglyEntanglingLayers(weights, wires=range(3))

    # Rotate wire 0
    qml.RY(-np.pi/2, wires=0)

    # Rotate wire 1
    qml.RY(np.pi/2, 1)

    return qml.probs(wires=range(3))

rotated_probs = circuit_qwc(weights)
print(qml.draw(circuit_qwc)(weights))

print(rotated_probs)

0: ─╭StronglyEntanglingLayers(M0)──RY(-1.57)─┤ ╭Probs
1: ─├StronglyEntanglingLayers(M0)──RY(1.57)──┤ ├Probs
2: ─╰StronglyEntanglingLayers(M0)────────────┤ ╰Probs

M0 = 
[[[ 0.05703565 -0.07449128 -0.16901955]
  [-0.12016696 -0.06550746 -0.01767452]
  [ 0.13384846  0.15537058 -0.1850643 ]]

 [[ 0.03739748 -0.11679195 -0.00849701]
  [ 0.08411111  0.17348948 -0.08573166]
  [-0.08463818  0.06356534  0.20450006]]

 [[-0.11389128  0.11884404  0.04941816]
  [-0.01232847 -0.03289872  0.06589769]
  [-0.01083437  0.00766881  0.09565878]]]
[0.28060674 0.00556428 0.28519516 0.00038902 0.17013419 0.01409715
 0.23999957 0.00401389]


In [7]:
# Single-qubit Pauli: [1,-1] , identity: [1,1]
eigenvalues_XYI = np.kron(np.kron([1, -1], [1, -1]), [1, 1])
eigenvalues_XIZ = np.kron(np.kron([1, -1], [1, 1]), [1, -1])

print(eigenvalues_XYI)
print(eigenvalues_XIZ)

# Taking the linear combination of the eigenvalues and the probabilities
print("Expectation value of XYI = ", np.dot(eigenvalues_XYI, rotated_probs))
print("Expectation value of XIZ = ", np.dot(eigenvalues_XIZ, rotated_probs))

[ 1  1 -1 -1 -1 -1  1  1]
[ 1 -1  1 -1 -1  1 -1  1]
Expectation value of XYI =  0.060368966729053244
Expectation value of XIZ =  0.1678258917705898


In [14]:
ev_IZ = np.kron([1,1], [1,-1])
ev_ZI = np.kron([1,-1], [1,1])
ev_ZZ = np.kron([1,-1], [1,-1])
ev_YY = np.kron([1,-1], [1,-1])
ev_XX = np.kron([1,-1], [1,-1])

print(ev_IZ)
print(ev_ZI)
print(ev_ZZ)
print(ev_YY)
print(ev_XX)

[ 1 -1  1 -1]
[ 1  1 -1 -1]
[ 1 -1 -1  1]
[ 1 -1 -1  1]
[ 1 -1 -1  1]
